In [1]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import LSTM
from keras.layers import Input
from functools import partial
from keras.models import Model
import tensorflow as tf

In [2]:
import utils
import processing as pr

In [3]:
class RandomWeightedAverage(Layer):
    def _merge_function(self, inputs):
        alpha = K.random_uniform((64,1,1))
        return (alpha)*inputs[0] + (1-alpha)*inputs[1]

In [4]:
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

In [5]:
def gradient_penalty_loss(y_true, y_pred, averaged_samples):
    gradients = K.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    gradient_penalty = K.square(1 - gradient_l2_norm)
    return K.mean(gradient_penalty)

In [6]:
def build_encoder_layer(input_shape, encoder_reshape_shape):

    input_layer = layers.Input(shape=(input_shape))

    x = layers.Bidirectional(LSTM(units=100, return_sequences=True))(input_layer)
    x = layers.Flatten()(x)
    x = layers.Dense(encoder_reshape_shape[0]*encoder_reshape_shape[1])(x)
    x = layers.Reshape(target_shape=encoder_reshape_shape)(x)
    model = keras.models.Model(input_layer, x, name='encoder')

    return model

In [7]:
def build_generator_layer(input_shape, generator_reshape_shape):

    input_layer = layers.Input(shape=input_shape)
    x = layers.Flatten()(input_layer)
    x = layers.Dense(generator_reshape_shape[0]*generator_reshape_shape[1])(x)
    x = layers.Reshape(target_shape=generator_reshape_shape)(x)
    x = layers.Bidirectional(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat')(x)
    x = layers.UpSampling1D(size=2)(x)
    x = layers.Bidirectional(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat')(x)
    x = layers.TimeDistributed(layers.Dense(input_shape[-1]))(x)
    x = layers.Activation(activation='tanh')(x)
    model = keras.models.Model(input_layer, x, name='generator')
    
    return model

In [8]:
def build_critic_x_layer(input_shape):

    input_layer = layers.Input(shape=input_shape)

    x = layers.Conv1D(filters=64, kernel_size=5)(input_layer)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.Conv1D(filters=64, kernel_size=5)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.Conv1D(filters=64, kernel_size=5)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.Conv1D(filters=64, kernel_size=5)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units=1)(x)
    model = keras.models.Model(input_layer, x, name='critic_x')

    return model

In [9]:
def build_critic_z_layer(input_shape):

    input_layer = layers.Input(shape=input_shape)

    x = layers.Flatten()(input_layer)
    x = layers.Dense(units=100)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.2)(x)
    x = layers.Dense(units=100)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.2)(x)
    x = layers.Dense(units=1)(x)
    model = keras.models.Model(input_layer, x, name='critic_z')

    return model

In [10]:
seqs = 100
channels = 38
latent_dim = 20
shape = (seqs,channels)
target_shape = (seqs,channels)

encoder_input_shape = (seqs, channels)
generator_input_shape = (seqs//5, channels)
critic_x_input_shape = (seqs, channels)
critic_z_input_shape = (seqs//5, channels)
encoder_reshape_shape = (seqs//5, channels)
generator_reshape_shape = (seqs//2, channels)

In [11]:
optimizer = tf.keras.optimizers.Adam(0.0005)

In [12]:
encoder = build_encoder_layer(input_shape=encoder_input_shape, encoder_reshape_shape=encoder_reshape_shape)
generator = build_generator_layer(input_shape=generator_input_shape, generator_reshape_shape=generator_reshape_shape)
critic_x = build_critic_x_layer(input_shape=critic_x_input_shape)
critic_z = build_critic_z_layer(input_shape=critic_z_input_shape)

In [13]:
generator.trainable = False
encoder.trainable = False

In [14]:
x = Input(shape=shape)
y = Input(shape=target_shape)
z = Input(shape=(latent_dim, channels))

In [15]:
x_ = generator(z)
z_ = encoder(x)
fake_x = critic_x(x_)
valid_x = critic_x(y)

In [16]:
# interpolated_x = RandomWeightedAverage()([y, x_])
alpha1 = K.random_uniform((64,1,1))
interpolated_x = y*alpha1 + x_*(1-alpha1)
validity_interpolated_x = critic_x(interpolated_x)
partial_gp_loss_x = partial(gradient_penalty_loss, averaged_samples=interpolated_x)
partial_gp_loss_x.__name__ = 'gradient_penalty'
critic_x_model = Model(inputs=[y, z], outputs=[valid_x, fake_x,
                                                    validity_interpolated_x])
critic_x_model.compile(loss=[wasserstein_loss, wasserstein_loss,
                                    partial_gp_loss_x], optimizer=optimizer,
                            loss_weights=[1, 1, 10])

In [17]:
fake_z = critic_z(z_)
valid_z = critic_z(z)
alpha2 = K.random_uniform((64,1,1))
# interpolated_z = RandomWeightedAverage()([z, z_])
interpolated_z = z*alpha2 + z_*(1-alpha2)
validity_interpolated_z = critic_z(interpolated_z)
partial_gp_loss_z = partial(gradient_penalty_loss, averaged_samples=interpolated_z)
partial_gp_loss_z.__name__ = 'gradient_penalty'
critic_z_model = Model(inputs=[x, z], outputs=[valid_z, fake_z,
                                                    validity_interpolated_z])
critic_z_model.compile(loss=[wasserstein_loss, wasserstein_loss,
                                    partial_gp_loss_z], optimizer=optimizer,
                            loss_weights=[1, 1, 10])

In [18]:
critic_x.trainable = False
critic_z.trainable = False
generator.trainable = True
encoder.trainable = True

In [19]:
z_gen = Input(shape=(latent_dim, channels))
x_gen_ = generator(z_gen)
x_gen = Input(shape=shape)
z_gen_ = encoder(x_gen)
x_gen_rec = generator(z_gen_)
fake_gen_x = critic_x(x_gen_)
fake_gen_z = critic_z(z_gen_)

In [20]:
encoder_generator_model = Model([x_gen, z_gen], [fake_gen_x, fake_gen_z, x_gen_rec])
encoder_generator_model.compile(loss=[wasserstein_loss, wasserstein_loss,
                                            'mse'], optimizer=optimizer,
                                        loss_weights=[1, 1, 10])

In [21]:
batch_size = 64
epochs = 35
iterations_critic = 5

In [22]:
dataloader = pr.data_loader()

train_set, test_set, label_set = dataloader.SMD()

train_set, test_set = dataloader.min_max(train_set, test_set)

train_set_overlap = dataloader.window_overlap(train_set,100,0)
test_set_overlap = dataloader.window_overlap(test_set,100,0)

In [23]:
X = train_set_overlap
target = train_set_overlap

In [24]:
fake = np.ones((batch_size, 1))
valid = -np.ones((batch_size, 1))
delta = np.ones((batch_size, 1))

indices = np.arange(X.shape[0])
for epoch in range(1, epochs + 1):
    np.random.shuffle(indices)
    X_ = X[indices]
    y_ = target[indices]

    epoch_g_loss = []
    epoch_cx_loss = []
    epoch_cz_loss = []

    minibatches_size = batch_size * iterations_critic
    num_minibatches = int(X_.shape[0] // minibatches_size)

    for i in range(num_minibatches):
        minibatch = X_[i * minibatches_size: (i + 1) * minibatches_size]
        y_minibatch = y_[i * minibatches_size: (i + 1) * minibatches_size]

        for j in range(iterations_critic):
            x = minibatch[j * batch_size: (j + 1) * batch_size]
            y = y_minibatch[j * batch_size: (j + 1) * batch_size]
            z = np.random.normal(size=(batch_size, latent_dim, channels))
            epoch_cx_loss.append(
                critic_x_model.train_on_batch([y, z], [valid, fake, delta]))
            epoch_cz_loss.append(
                critic_z_model.train_on_batch([x, z], [valid, fake, delta]))

        epoch_g_loss.append(
            encoder_generator_model.train_on_batch([x, z], [valid, valid, y]))

    cx_loss = np.mean(np.array(epoch_cx_loss), axis=0)
    cz_loss = np.mean(np.array(epoch_cz_loss), axis=0)
    g_loss = np.mean(np.array(epoch_g_loss), axis=0)
    print('Epoch: {}/{}, [Dx loss: {}] [Dz loss: {}] [G loss: {}]'.format(
        epoch, epochs, cx_loss, cz_loss, g_loss))

TypeError: in user code:

    File "c:\Users\minseok\anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\minseok\AppData\Local\Temp/ipykernel_25056/1252378524.py", line 2, in gradient_penalty_loss  *
        gradients = K.gradients(y_pred, averaged_samples)[0]
    File "c:\Users\minseok\anaconda3\lib\site-packages\keras\backend.py", line 4263, in gradients  **
        return tf.compat.v1.gradients(
    File "c:\Users\minseok\anaconda3\lib\site-packages\numpy\core\_asarray.py", line 102, in asarray
        return array(a, dtype, copy=False, order=order)
    File "c:\Users\minseok\anaconda3\lib\site-packages\keras\engine\keras_tensor.py", line 255, in __array__
        raise TypeError(

    TypeError: You are passing KerasTensor(type_spec=TensorSpec(shape=(64, 100, 38), dtype=tf.float32, name=None), name='tf.__operators__.add/AddV2:0', description="created by layer 'tf.__operators__.add'"), 